# 椭圆曲线加密

考虑一条椭圆曲线$E_p:y^2=x^3+ax+b(modp)$，其上有点G，G的阶为n（$nG=\infty$）。任选整数$1<k<n$，计算$K=kG$，则整数k和点K，被称为一对密钥，k为私钥，K为公钥，点G被称为基点。

secp256k1椭圆曲线参数如下：

In [1]:
from simchain.ecc import secp256k1

2019-04-28 19:07:05,766 - Loaded backend module://ipykernel.pylab.backend_inline version unknown.


In [2]:
print("a =", secp256k1.curvefp.a)
print("b =", secp256k1.curvefp.b)
print("p =", secp256k1.curvefp.p)
# 基点G
print("G =", secp256k1.generator)
# 基点G的阶
print("G order =", secp256k1.order)

a = 0
b = 7
p = 115792089237316195423570985008687907853269984665640564039457584007908834671663
G = (55066263022277343669578718895168534326250603453777594175500187360389116729240,32670510020758816978083085130507043184471273380659243275938904335757337482424)
G order = 115792089237316195423570985008687907852837564279074904382605163141518161494337


## 公钥和私钥的生成
随机选择一个小于n的整数作为私钥，并计算与之对应的公钥

In [3]:
import random
# 生成一个私钥
k = random.randint(1, secp256k1.order)
print("k =", k)
# 计算公钥
K = secp256k1.generator * k
print("K =", K)

k = 13721621457836194048043572723245894404046164988136268389994891683815006016465
K = (74340034498203696003321400411194848503900624738181122002403246597948302981611,100299871651858520895354144543649521888584284064794546885766123016609593764869)


显示密钥：将整数编码成固定长度的字节串

In [4]:
from simchain import SigningKey, VerifyingKey, secp256k1
# 随机生成私钥整数
k = random.randint(1, secp256k1.order)
# 通过整数创建私钥对象
sk = SigningKey.from_number(k)
# 私钥编码成字节串显示
sk.to_bytes()

b"\x1f3\xfd=\xe0nPW\xdf\xf8'\x17\x89\x9aQ\x17\x1d\x90cv?W5\r\xb8\xab\xe1\xa6K\xa3\xd5\x08"

In [5]:
# 获取该私钥对应的公钥对象
pk = sk.get_verifying_key()
# 将公钥编码成字节串显示
pk.to_bytes()

b'\xe1K\xc0\x97\x01\xf1\xfds\xab\xf4\xb9\x87\xd2\xdd\xc8\x0c\xbf\xc3r\x10$\r\xe7-\xcc\xa7\xc5\r.6\x9e\xf2V"\xfb\x93\x91i\xdai\xf1-&\x19\xf8\xa1\xb2S\xf1\xe1\xd2\r\xf3\xdc-nm\xf4:\xb8\x00\xa8\xce\xf0'

In [6]:
# 查看私钥的数值
k

14113555316402774164935292687436306854491657195741066780265585112207665124616

In [7]:
# 导入解码函数
from simchain.ecc import bytes_to_number
# 将私钥从字节串转换成整数
bytes_to_number(sk.to_bytes())

14113555316402774164935292687436306854491657195741066780265585112207665124616

In [8]:
# 将公钥从字节串转换成整数对
(bytes_to_number(pk.to_bytes()[0:32]), bytes_to_number(pk.to_bytes()[32:]))

(101904233667548707959365884539969791313808988891615054231382830273216745217778,
 38960714095122406734039837441172690209171861768897369501727383017946362728176)

In [9]:
# 直接计算公钥
secp256k1.generator * k

(101904233667548707959365884539969791313808988891615054231382830273216745217778,38960714095122406734039837441172690209171861768897369501727383017946362728176)

In [10]:
# 由字节串得到公钥对象
ppk = VerifyingKey.from_bytes(pk.to_bytes())
ppk.to_bytes()

b'\xe1K\xc0\x97\x01\xf1\xfds\xab\xf4\xb9\x87\xd2\xdd\xc8\x0c\xbf\xc3r\x10$\r\xe7-\xcc\xa7\xc5\r.6\x9e\xf2V"\xfb\x93\x91i\xdai\xf1-&\x19\xf8\xa1\xb2S\xf1\xe1\xd2\r\xf3\xdc-nm\xf4:\xb8\x00\xa8\xce\xf0'

## 地址的生成
本书的地址采用与比特币一样的算法：
1. 对公钥字节串进行两次哈希运算，分别采用sha256和ripemd160哈希算法，得到一个160位的公钥哈希值；
2. 然后在其前面加上一个字节的地址版本信息\x00，得到一个21字节的字节串；
3. 然后对该字节串采用base58编码成固定长度的字符串，得到地址。

In [14]:
# 从内置哈希运算库中导入new和sha256对象
from hashlib import new, sha256
# 从base58模块中调用编码函数
from base58 import b58encode_check

# 定义由公钥字节串生成地址的函数
def convert_pubkey_to_addr(pubkey_str):
    # 对字节串进行sha256哈希运算
    sha = sha256(pubkey_str).digest()
    # 进行ripemd160哈希运算
    ripe = new('ripemd160', sha).digest()
    # 进行base58编码
    return b58encode_check(b'\x00' + ripe).decode()

In [15]:
convert_pubkey_to_addr(ppk.to_bytes())

'1AekCNLaq1WeJDSkcWDMTDy61CY1VLWPe'

## 数字签名与验证
假设某数字货币采用椭圆曲线加密，已知选用的椭圆曲线为$E_p(a,b)$，基点为$G$。
### 数字签名
如果张三要创建一条有效交易，如何用单个UTXO创建每一个输入单元？  
1. 在区块链中找到指向自己地址A的UTXO。
2. 将交易的输入单元指向该UTXO，即定位指针指向该条UTXO在区块链中的位置，同时将自己钱包中与地址A对应的公钥K放入其中。
3. 张三选择一个随机数$rk$，一条公开明文$m$，将公开明文$m$的哈希值转换成整数$h$。
4. 计算点$rG=rk\bullet G$，并令$r=rG.x$，然后计算$s=(h+k\bullet r)/rk$，其中$k$为与公钥$K$对应的私钥。
5. 张三将$r,s$放到输入单元中。

### 验证签名
1. 李四将输入单元中的公钥$K$转换成地址$A'$，如果$A'=A$，执行下一步。
2. 李四将公开明文$m$用与张三一样的方式转换成整数$K$。
3. 李四计算点$P=h \bullet G/s+r \bullet K/s$
4. 判断$r=P.x$，为真则验证通过，否则验证不通过。

In [21]:
from random import SystemRandom
# 数字签名函数，输入为签名明文message，基点G，私钥k
def sign(message,G,k):
    # 获得基点G的阶
    n = G.order
    
    # 计算明文哈希值
    mess_hash = sha256(message).digest()
    
    # 将明文哈希值转换成数字
    h = bytes_to_number(mess_hash)
    r, s, = 0, 0
    while r == 0 or s == 0:
        # 生成随机数rk
        rk = SystemRandom().randrange(1, n)
        rG = rk*G
        r = rG.x
        s = ((h + (r*k)%n)*inv_mod(rk, n)) % n
    return r,s

In [22]:
# 验证签名函数，输入为签名sig，基点G，公钥K，以及明文message
def verify(sig,G,K,message):
    # 获取签名
    r,s = sig
    # 获取基点的阶
    n = G.order
    mess_hash = sha256(message).digest()
    h = bytes_to_number(mess_hash)
    w = inv_mod(s,n)
    u1, u2 = (h * w) % n,(r * w) % n
    p = u1 * G + u2 * K
    return r == p.x % n 

In [25]:
from simchain.ecc import secp256k1, sign, verify

# 获取基点G
G = secp256k1.generator
# 选择明文
message = b"I love blockchain"
# 选择私钥整数
k = 12345
# 计算对应的公钥整数对
K = k*G
print("K =", K)

K = (108607064596551879580190606910245687803607295064141551927605737287325610911759,6661302038839728943522144359728938428925407345457796456954441906546235843221)


In [26]:
# 用私钥签名
signature = sign(message, G, k)
print("signature =", signature)

signature = (43169889414249851771786216207002742298422049365183073422013828316769701709071, 64956080845433340310512840938058462946302782351993759365079085827841721408798)


In [28]:
# 用公钥验证签名
flag = verify(signature, G, K, message)
assert flag == True

尝试用一个新的私钥进行签名

In [30]:
# 新的私钥
k1 = 123456
# 新的签名
signature1 = sign(message, G, k1)
# 旧公钥验证
flag = verify(signature1, G, K, message)
# 验证签名不通过
assert flag == False